# VAE + GAN
https://arxiv.org/pdf/1512.09300.pdf

Попробуем улучшить генерацию за счет применения GAN

$X$ - входной объект из $X$

$Z_p$ - сэмплированный $Z$ из $P(Z)$

$X_p$ - объект, сгенерированный декодером из $Z_p$

$\tilde{X}$ - объект, восстановленный из $X$

$L_{prior} = D_{KL}(Q(Z|X) || P(Z))$ - лосс, заставляющий энкодер переводить $P(X)$ в нужное нам $P(Z)$

$L_{llike}^{Dis_l} = L_d(d_l(X), d_l(\tilde{X}))$ - лосс, между активациями l-ого слоя дискриминатора $D$ на реальном $X$ и восстановленном $\tilde{X}$

$L_{GAN} = log(D(X)) + log(1 - D(f_d(Z))) + log(1 - D(f_d(Q(X))))$ - кросс-энтропия между реальным распределением лейблов настоящих/сгенерированных объектов, и распределнием вероятности, предсказываемым дискриминатором.

Схема, предлагаемая авторами:

$\theta_{Enc} = \theta_{Enc} - \Delta_{\theta_{Enc}}(L_{prior} + L_{llike}^{Dis_l})$

$\theta_{Dec} = \theta_{Dec} - \Delta_{\theta_{Dec}}(\gamma L_{llike}^{Dis_l} - L_{GAN})$

$\theta_{Dis} = \theta_{Dis} - \Delta_{\theta_{Dis}}(L_{GAN})$
m
Каждую из трех моделей нужно учить по очереди.

# Подключение необходимых модулей

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import Subset
from torch.autograd import Variable


import matplotlib.pyplot as plt
%matplotlib inline

from VAEGANcfg import device, batch_size, epochs, lr, alpha, gamma
from VAEGAN import VAE_GAN, Discriminator

print(f'Selected device is {device}')

Selected device is cuda:0


# Загрузка датасета MNIST

In [2]:
transform=transforms.Compose([ transforms.Resize(64),transforms.CenterCrop(64),transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])
dataset=MNIST(root=".", train=True,transform=transform, download=True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


## Определение модели

In [5]:
gen=VAE_GAN().to(device)
discrim=Discriminator().to(device)
real_batch = next(iter(data_loader))

In [7]:
criterion=nn.BCELoss().to(device)
optim_E=torch.optim.RMSprop(gen.encoder.parameters(), lr=lr)
optim_D=torch.optim.RMSprop(gen.decoder.parameters(), lr=lr)
optim_Dis=torch.optim.RMSprop(discrim.parameters(), lr=lr*alpha)
z_fixed=Variable(torch.randn((64,128))).to(device)
x_fixed=Variable(real_batch[0]).to(device)

## Цикл обучения

In [8]:
for epoch in range(epochs):
    prior_loss_list,gan_loss_list,recon_loss_list=[],[],[]
    dis_real_list,dis_fake_list,dis_prior_list=[],[],[]
    for i, (data,_) in enumerate(data_loader, 0):
        bs=data.size()[0]

        ones_label=Variable(torch.ones(bs,1)).to(device)
        zeros_label=Variable(torch.zeros(bs,1)).to(device)
        zeros_label1=Variable(torch.zeros(64,1)).to(device)
        datav = Variable(data).to(device)
        mean, logvar, rec_enc = gen(datav)
        z_p = Variable(torch.randn(64,128)).to(device)
        x_p_tilda = gen.decoder(z_p)

        output = discrim(datav)[0]
        errD_real = criterion(output, ones_label)
        dis_real_list.append(errD_real.item())
        output = discrim(rec_enc)[0]
        errD_rec_enc = criterion(output, zeros_label)
        dis_fake_list.append(errD_rec_enc.item())
        output = discrim(x_p_tilda)[0]
        errD_rec_noise = criterion(output, zeros_label1)
        dis_prior_list.append(errD_rec_noise.item())
        gan_loss = errD_real + errD_rec_enc + errD_rec_noise
        gan_loss_list.append(gan_loss.item())
        optim_Dis.zero_grad()
        gan_loss.backward(retain_graph=True)
        optim_Dis.step()


        output = discrim(datav)[0]
        errD_real = criterion(output, ones_label)
        output = discrim(rec_enc)[0]
        errD_rec_enc = criterion(output, zeros_label)
        output = discrim(x_p_tilda)[0]
        errD_rec_noise = criterion(output, zeros_label1)
        gan_loss = errD_real + errD_rec_enc + errD_rec_noise


        x_l_tilda = discrim(rec_enc)[1]
        x_l = discrim(datav)[1]
        rec_loss = ((x_l_tilda - x_l) ** 2).mean()
        err_dec = gamma * rec_loss - gan_loss 
        recon_loss_list.append(rec_loss.item())
        optim_D.zero_grad()
        err_dec.backward(retain_graph=True)
        optim_D.step()

        mean, logvar, rec_enc = gen(datav)
        x_l_tilda = discrim(rec_enc)[1]
        x_l = discrim(datav)[1]
        rec_loss = ((x_l_tilda - x_l) ** 2).mean()
        prior_loss = 1 + logvar - mean.pow(2) - logvar.exp()
        prior_loss = (-0.5 * torch.sum(prior_loss))/torch.numel(mean.data)
        prior_loss_list.append(prior_loss.item())
        err_enc = prior_loss + 5*rec_loss

        optim_E.zero_grad()
        err_enc.backward(retain_graph=True)
        optim_E.step()

        if i % 50 == 0:
                print('[%d/%d][%d/%d]\tLoss_gan: %.4f\tLoss_prior: %.4f\tRec_loss: %.4f\tdis_real_loss: %0.4f\tdis_fake_loss: %.4f\tdis_prior_loss: %.4f'
                      % (epoch,epochs, i, len(data_loader),
                         gan_loss.item(), prior_loss.item(),rec_loss.item(),errD_real.item(),errD_rec_enc.item(),errD_rec_noise.item()))


    b=gen(x_fixed)[2]
    b=b.detach()
    c=gen.decoder(z_fixed)
    c=c.detach()
    #show_and_save('MNISTrec_noise_epoch_%d.png' % epoch ,make_grid((c*0.5+0.5).cpu(),8))
    #show_and_save('MNISTrec_epoch_%d.png' % epoch ,make_grid((b*0.5+0.5).cpu(),8))

#plot_loss(prior_loss_list)
#plot_loss(recon_loss_list)
#plot_loss(gan_loss_list)

[0/25][0/938]	Loss_gan: 2.8758	Loss_prior: 0.1302	Rec_loss: 0.7129	dis_real_loss: 1.0043	dis_fake_loss: 0.8517	dis_prior_loss: 1.0198


KeyboardInterrupt: 